# Query generation

# Self metadata query

## Ingestion (metadata enriched)

### Preparing the Chroma DB collection

In [44]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import getpass

OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY')

Enter your OPENAI_API_KEY ········


In [2]:
uk_with_metadata_collection = Chroma(
    collection_name="uk_with_metadata_collection",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_with_metadata_collection.reset_collection() #in case it already exists

### Setting up the RecursiveCharacterTextSplitter 

In [3]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
html2text_transformer = Html2TextTransformer()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)

In [6]:
def split_docs_into_chunks(docs):
    text_docs = html2text_transformer.transform_documents(docs) #A 
    chunks = text_splitter.split_documents(text_docs)

    return chunks
#A transform HTML docs into clean text docs

In [7]:
uk_destinations = [
    ("Cornwall", "Cornwall"), ("North_Cornwall", "Cornwall"), 
    ("South_Cornwall", "Cornwall"), ("West_Cornwall", "Cornwall"),
    ("Tintagel", "Cornwall"), ("Bodmin", "Cornwall"), ("Wadebridge", "Cornwall"),
    ("Penzance", "Cornwall"), ("Newquay", "Cornwall"), ("St_Ives", "Cornwall"),
    ("Port_Isaac", "Cornwall"), ("Looe", "Cornwall"), ("Polperro", "Cornwall"),
    ("Porthleven", "Cornwall"),
    ("East_Sussex", "East_Sussex"), ("Brighton", "East_Sussex"),
    ("Battle", "East_Sussex"), ("Hastings_(England)", "East_Sussex"),
    ("Rye_(England)", "East_Sussex"), ("Seaford", "East_Sussex"), 
    ("Ashdown_Forest", "East_Sussex")
]

wikivoyage_root_url = "https://en.wikivoyage.org/wiki"

In [8]:
uk_destination_url_with_metadata = [
    ( f'{wikivoyage_root_url}/{destination}', destination, region)
    for destination, region in uk_destinations]

### Enriching a document with metadata: updating metadata 

In [9]:
tintagel_url, tintagel_destination, tintagel_region = uk_destination_url_with_metadata[4]

In [10]:
tintagel_html_loader =AsyncHtmlLoader(tintagel_url)
tintagel_docs = tintagel_html_loader.load()

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.18it/s]


In [11]:
# tintagel_docs # COMMENT: LangChain loaders create docs which contain metadata

In [12]:
for doc in tintagel_docs:
    doc.metadata['destination'] = tintagel_destination
    doc.metadata['region'] = tintagel_region
    print(doc.metadata)

{'source': 'https://en.wikivoyage.org/wiki/Tintagel', 'title': 'Tintagel – Travel guide at Wikivoyage', 'language': 'en', 'destination': 'Tintagel', 'region': 'Cornwall'}


### Enriching a document with metadata: creating metadata 

In [13]:
tintagel_docs_with_metadata = [
    Document(page_content=d.page_content,
             metadata = {
                 'source': tintagel_url,
                 'destination': tintagel_destination,
                 'region': tintagel_region
             })
    for d in tintagel_docs
]

In [14]:
# tintagel_docs_with_metadata # examine the Document

### Enriching the UK destination documents with metadata: creating metadata 

In [15]:
for (url, destination, region) in uk_destination_url_with_metadata:
    html_loader = AsyncHtmlLoader(url) #A
    docs =  html_loader.load() #B
    
    docs_with_metadata = [
        Document(page_content=d.page_content,
        metadata = {
            'source': url,
            'destination': destination,
            'region': region})
        for d in docs]
             
    chunks = split_docs_into_chunks(docs_with_metadata)

    print(f'Importing: {destination}')
    uk_with_metadata_collection.add_documents(documents=chunks)
#A Loader for one destination
#B Documents of one destination 

Fetching pages: 100%|####################################################################| 1/1 [00:01<00:00,  1.18s/it]


Importing: Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.69it/s]


Importing: North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:01<00:00,  1.39s/it]


...

Importing: Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.74it/s]


Importing: Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.18it/s]


Importing: Ashdown_Forest


## Q & A on a collection enriched with metadata

### Searching the collection with a metadata filter explicitly

In [16]:
question =  "Events or festivals"
metadata_retriever = uk_with_metadata_collection.as_retriever(search_kwargs={'k':1, 'filter':{'destination': 'Newquay'}})

result_docs = metadata_retriever.invoke(question)

In [17]:
result_docs

[Document(metadata={'destination': 'Newquay', 'region': 'Cornwall', 'source': 'https://en.wikivoyage.org/wiki/Newquay'}, page_content="## Do\n\n[edit]\n\n  * Cornish Film Festival. Held annually for two weeks each November around Newquay. (updated Jan 2024)\n  * 50.415741-5.0914781 Newquay Golf Club, Tower Road, TR7 1LT, ☏ +44 1637 872091, info@newquaygolfclub.co.uk. 9AM-4PM. A semi-private golf club established in 1890. Total yardage Championship: 6141, Men: 5708, and Women: 5364. £31 for non-members. (updated Apr 2019)\n\n### Beaches\n\n[edit]\n\nFistral Beach\n\nNewquay is well known as a surfer's paradise. Therefore it offers plenty of\nbeaches:")]

In [18]:
# COMMENT: As you can see, only chunks associated with'destination': 'Newquay' have been selected

### Generating the self metadata query with the SelfQueryRetriever

In [19]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever #A
from langchain_openai import ChatOpenAI
#A this requires pip install lark

In [20]:
metadata_field_info = [
    AttributeInfo(
        name="destination",
        description="The specific UK destination to be searched",
        type="string",
    ),
    AttributeInfo(
        name="region",
        description="The name of the UK region to be searched",
        type="string",
    )
]

In [21]:
question = "Tell me about events or festivals in the UK town of Newquay"

In [22]:
document_content_description = "Brief summary of a movie"
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

self_query_retriever = SelfQueryRetriever.from_llm(
    llm, uk_with_metadata_collection, question, metadata_field_info, verbose=True
)

In [23]:
result_docs = self_query_retriever.invoke(question)

In [24]:
result_docs

[Document(metadata={'destination': 'Cornwall', 'region': 'Cornwall', 'source': 'https://en.wikivoyage.org/wiki/Cornwall'}, page_content="The **Cornish Film Festival** is held annually each November around Newquay.\n\nCornwall, in particular Newquay, is the UK's **surfing** capital, with\nequipment hire and surf schools present on many of the county's beaches, and\nevents like the UK championships or Boardmasters festival.\n\n## Eat\n\n[edit]\n\nCornwall has become famous for its Michelin-starred seafood restaurants, with\nJamie Oliver and Rick Stein opening swanky restaurants in the county/country.\nCornwall may have the most distinct and finest cuisine of all Britain, and a\nnumber of regional specialities, such as:\n\n### Savoury\n\n[edit]\n\nCornish Pasty"),
 Document(metadata={'destination': 'North_Cornwall', 'region': 'Cornwall', 'source': 'https://en.wikivoyage.org/wiki/North_Cornwall'}, page_content='The **Cornish Film Festival** is held annually each November around Newquay.\n\

### Generating the self metadata query with a LLM function call

#### Query schema

In [25]:
import datetime
from typing import Literal, Optional, Tuple, List

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
)
from langchain.retrievers.self_query.chroma import ChromaTranslator

In [26]:
class DestinationSearch(BaseModel):
    """Search over a vector database of tourist destinations."""

    content_search: str = Field(
        "",
        description="Similarity search query applied to tourist destinations.",
    )
    destination: str = Field(
        ...,
        description="The specific UK destination to be searched.",
    )
    region: str = Field(
        ...,
        description="The name of the UK region to be searched.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

In [27]:
def build_filter(destination_search: DestinationSearch):
    comparisons = []

    destination = destination_search.destination #A
    region = destination_search.region #A
    
    if destination and destination != '': #B
        comparisons.append(
            Comparison(
                comparator=Comparator.EQ,
                attribute="destination",
                value=destination,
            )
        )
    if region and region != '': #C
        comparisons.append(
            Comparison(
                comparator=Comparator.EQ,
                attribute="region",
                value=region,
            )
        )    

    search_filter = Operation(operator=Operator.AND, arguments=comparisons) #D

    chroma_filter = ChromaTranslator().visit_operation(search_filter) #E
        
    return chroma_filter
#A Get destination and region from the structured query
#B If the destination exists, create an 'equality' operation
#C If the region exists, create an 'equality' operation
#D Create a combined search filter
#E Transform the filter into Chroma format

#### Conversion of user question to structured query including metadata filter

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system_message = """You are an expert at converting user questions into vector database queries. \
You have access to a database of tourist destinations. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)
structured_llm = llm.with_structured_output(DestinationSearch)
query_generator = prompt | structured_llm

In [29]:
question = "Tell me about events or festivals in the UK town of Newquay"

structured_query =query_generator.invoke(question)

In [30]:
structured_query

DestinationSearch(content_search='events festivals', destination='Newquay', region='Cornwall')

In [31]:
search_filter = build_filter(structured_query)

In [32]:
search_filter

{'$and': [{'destination': {'$eq': 'Newquay'}},
  {'region': {'$eq': 'Cornwall'}}]}

In [33]:
search_query = structured_query.content_search

In [34]:
search_query

'events festivals'

In [35]:
metadata_retriever = uk_with_metadata_collection.as_retriever(search_kwargs={'k':3, 'filter': search_filter})

In [36]:
answer = metadata_retriever.invoke(search_query)

In [37]:
print(answer)

[Document(metadata={'destination': 'Newquay', 'region': 'Cornwall', 'source': 'https://en.wikivoyage.org/wiki/Newquay'}, page_content="## Do\n\n[edit]\n\n  * Cornish Film Festival. Held annually for two weeks each November around Newquay. (updated Jan 2024)\n  * 50.415741-5.0914781 Newquay Golf Club, Tower Road, TR7 1LT, ☏ +44 1637 872091, info@newquaygolfclub.co.uk. 9AM-4PM. A semi-private golf club established in 1890. Total yardage Championship: 6141, Men: 5708, and Women: 5364. £31 for non-members. (updated Apr 2019)\n\n### Beaches\n\n[edit]\n\nFistral Beach\n\nNewquay is well known as a surfer's paradise. Therefore it offers plenty of\nbeaches:"), Document(metadata={'destination': 'Newquay', 'region': 'Cornwall', 'source': 'https://en.wikivoyage.org/wiki/Newquay'}, page_content='## See\n\n[edit]\n\n  * 50.414578-5.0848411 Blue Reef Aquarium, Towan Promenade, TR7 1DU (right next to Towan beach), ☏ +44 1637 878134. Although it is small, it is well worth checking out. It has an octop

In [82]:
## COMMENT: this is only the retrieval step; you still need to wrap it in a RAG chain

# Generating a structured SQL query

## Connecting to the UkBooking database

In [115]:
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import getpass
import os

In [45]:
db = SQLDatabase.from_uri("sqlite:///UkBooking.db")
print(db.get_usable_table_names())

['Accommodation', 'AccommodationType', 'Booking', 'Customer', 'Destination', 'Offer']


In [46]:
db.run("SELECT * FROM Offer;")

"[(1, 1, 'Summer Special', 0.15, '2024-06-01', '2024-08-31'), (2, 2, 'Weekend Getaway', 0.1, '2024-09-01', '2024-12-31'), (3, 3, 'Early Bird Discount', 0.2, '2024-05-01', '2024-06-30'), (4, 4, 'Stay 3 Nights, Get 1 Free', 0.25, '2024-01-01', '2024-03-31'), (5, 5, 'Historic Stay Offer', 0.1, '2024-04-01', '2024-06-30'), (6, 6, 'Autumn Discount', 0.15, '2024-09-01', '2024-11-30'), (7, 7, 'Cottage Retreat Offer', 0.12, '2024-07-01', '2024-09-30'), (8, 8, 'City Break Deal', 0.08, '2024-10-01', '2024-12-31'), (9, 9, 'Luxury Villa Offer', 0.18, '2024-05-01', '2024-08-31'), (10, 10, 'Spa & Wellness Package', 0.2, '2024-04-01', '2024-07-31')]"

## Generate SQL queries from natural language

In [81]:
OPENAI_API_KEY = getpass.getpass()

 ········


### Generating the SQL query

In [102]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini", temperature=0)
sql_query_gen_chain = create_sql_query_chain(llm, db)
response = sql_query_gen_chain.invoke({"question": "Give me some offers for Cardiff, including the hotel name"})

In [103]:
response

'```sql\nSELECT "Offer"."OfferDescription", "Offer"."DiscountRate", "Accommodation"."Name" \nFROM "Offer" \nJOIN "Accommodation" ON "Offer"."AccommodationId" = "Accommodation"."AccommodationId" \nJOIN "Destination" ON "Accommodation"."DestinationId" = "Destination"."DestinationId" \nWHERE "Destination"."Name" = \'Cardiff\' \nLIMIT 5;\n```'

In [105]:
#db.run(response) # returns error

### Executing the SQL query [DOES NOT SEEM TO WORK: IF SO, SKIP IT]

In [104]:
sql_query_exec_chain = QuerySQLDataBaseTool(db=db)
sql_query_gen_chain = create_sql_query_chain(llm, db)
chain = sql_query_gen_chain | sql_query_exec_chain
chain.invoke({"question": "Give me some offers for Cardiff, including the hotel name"})

'Error: (sqlite3.OperationalError) near "```sql\nSELECT "Offer"."OfferDescription", "Offer"."DiscountRate", "Accommodation"."Name" \nFROM "Offer" \nJOIN "Accommodation" ON "Offer"."AccommodationId" = "Accommodation"."AccommodationId" \nJOIN "Destination" ON "Accommodation"."DestinationId" = "Destination"."DestinationId" \nWHERE "Destination"."Name" = \'Cardiff\' \nLIMIT 5;\n```": syntax error\n[SQL: ```sql\nSELECT "Offer"."OfferDescription", "Offer"."DiscountRate", "Accommodation"."Name" \nFROM "Offer" \nJOIN "Accommodation" ON "Offer"."AccommodationId" = "Accommodation"."AccommodationId" \nJOIN "Destination" ON "Accommodation"."DestinationId" = "Destination"."DestinationId" \nWHERE "Destination"."Name" = \'Cardiff\' \nLIMIT 5;\n```]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

### Fixing the SQL format

In [116]:
clean_sql_prompt_template = """You are an expert in SQL Lite. You are asked to fix badly formed SQL Lite queries, 
which might contain unneded prefixes or suffixes. Given the following unclean SQL statement, 
transform it to a clean, executable SQL statement for SQL lite.
Only return an executable SQL statement which terminates with a semicolon. Do not return anything else.
Do not include the language name or symbols like ```.

Unclean SQL: {unclean_sql}"""

In [117]:
clean_sql_prompt = ChatPromptTemplate.from_template(clean_sql_prompt_template)

In [118]:
clean_sql_chain = clean_sql_prompt | llm

In [120]:
full_sql_gen_chain = sql_query_gen_chain | clean_sql_chain | StrOutputParser()

In [190]:
question = "Give me some offers for Cardiff, including the accomodation name"

In [191]:
response = full_sql_gen_chain.invoke({"question": question})

In [192]:
response

"SELECT Offer.OfferDescription, Offer.DiscountRate, Accommodation.Name \nFROM Offer \nJOIN Accommodation ON Offer.AccommodationId = Accommodation.AccommodationId \nJOIN Destination ON Accommodation.DestinationId = Destination.DestinationId \nWHERE Destination.Name = 'Cardiff' \nLIMIT 5;"

In [193]:
### Comment: now SQL is fixed

### Executing the SQL query

In [233]:
sql_query_exec_chain = QuerySQLDataBaseTool(db=db)

In [234]:
sql_query_gen_and_exec_chain = full_sql_gen_chain | sql_query_exec_chain | StrOutputParser()

In [236]:
response = sql_query_gen_and_exec_chain.invoke({"question":question})

In [197]:
response

"[('Early Bird Discount', 0.2, 'Cardiff Camping')]"

In [ ]:
## COMMENT: this is only the retrieval step; you still need to wrap it in a RAG chain

# Query router

In [239]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain.schema.runnable import RunnableLambda

## Setting up the data retrievers

### Setting up the vector store retriever

In [242]:
tourist_info_retriever_chain = RunnableLambda(lambda x: x['question']) | uk_with_metadata_collection.as_retriever(search_kwargs={'k':2}) 

### Setting up the relational database retriever (Same as sql_query_gen_and_exec_chain above)

In [243]:
uk_accommodation_retriever_chain =  full_sql_gen_chain | sql_query_exec_chain | StrOutputParser()

## Setting up the query router

In [244]:
class RouteQuery(BaseModel):
    """Route a user question to the most relevant datasource."""

    datasource: Literal["tourist_info_store", "uk_booking_db"] = Field(
        ...,
        description="Given a user question, route it either to a tourist info vector store or a UK accomodation booking relational database.",
    )

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")
structured_llm_router = llm.with_structured_output(RouteQuery) #A
#B LLM with function call

### Setting up the question router chain

In [245]:
system = """You are an expert at routing a user question to a tourist info vector store 
or to an UK accommodation booking relational database.
The vector store contains tourist information about UK destinations.
Use the vectorstore for general toruist information questions on UK destinations. 
For questions about accommodation availability or booking, use the UK Booking database."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

### Testing the router chain

In [246]:
selected_data_source = question_router.invoke(
    {"question": "Have you got any offers in Brighton?"}
)

In [247]:
print(selected_data_source)

datasource='uk_booking_db'


In [248]:
selected_data_source = question_router.invoke(
    {"question": "Where are the best beaches in Cornwall?"}
)

In [249]:
print(selected_data_source)

datasource='tourist_info_store'


### Setting up the retriever chooser

In [278]:
retriever_chains = {
    'tourist_info_store': tourist_info_retriever_chain,
    'uk_booking_db': uk_accommodation_retriever_chain
}

def retriever_chooser(question):
    selected_data_source = question_router.invoke(
        {"question": question})

    return retriever_chains[selected_data_source.datasource]

In [279]:
chosen = retriever_chooser('Tell me about events or festivals in the UK town of Newquay') ####REMOVE

In [280]:
print(chosen)####REMOVE

first=RunnableLambda(lambda x: x['question']) last=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002260247BB50>, search_kwargs={'k': 2})


## Setting up the full RAG chain

In [253]:
from langchain_core.runnables import RunnablePassthrough

In [254]:
rag_prompt_template = """
Given a question and some context, answer the question.
If you get a structured context, like a tuple, try to infer the meaning of the components: 
typically they refer to accommodation offers, and the number is a percentage (0.2 means 20%).
If you do not the answer, just say I do not know.


Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template) 

def execute_rag_chain(question, chosen_retriever):
    full_rag_chain = (
        {
            "context": {"question": RunnablePassthrough()} | chosen_retriever,#A
            "question": RunnablePassthrough(),#B
        }
        | rag_prompt
        | llm
        | StrOutputParser()
    )
    #A The context is returned by the retriver after feeding to it the rewritten query
    #B This is the original user question

    return full_rag_chain.invoke(question)

## Executing the full RAG chain 

### Question on offers

In [261]:
question = 'Give me some offers for Cardiff, including the accommodation name'

chosen_retriever = retriever_chooser(question)

answer = execute_rag_chain(question, chosen_retriever)

In [263]:
print(answer)

One offer for Cardiff is the "Early Bird Discount" at Cardiff Camping, which provides a 20% discount.


### Question on tourist information

In [283]:
question_2 = 'Tell me about events or festivals in the UK town of Newquay'

chosen_retriever_2 = retriever_chooser(question_2)

answer2 = execute_rag_chain(question_2, chosen_retriever_2)

In [285]:
print(answer2)

In Newquay, the **Cornish Film Festival** is held annually each November. Additionally, it is known for being the UK's surfing capital, hosting events like the UK championships and the Boardmasters festival.
